<a href="https://colab.research.google.com/github/gabrielborja/parc_de_montjuic/blob/main/cleaning_scripts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cleaning scripts

## Importing libraries

In [ ]:
# Upgrade Matplotlib
!pip install matplotlib --upgrade
#!pip install plotly --upgrade

In [ ]:
# Importing python libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
#from geopy.geocoders import Nominatim
from ipywidgets import interact

## Loading Data

In [ ]:
# Loading data from local drive
from google.colab import files
uploaded1 = files.upload()

In [ ]:
# Storing loaded data from excel to a pandas dataframe
import io
df1 = pd.read_excel(io.BytesIO(uploaded1['05_hline.xlsx']))

In [ ]:
# Storing loaded data from csv to a pandas dataframe
#import io
#df1 = pd.read_csv(io.BytesIO(uploaded1['20230504mdrift.csv']), sep='|', engine='python')#, encoding='latin-1')
#quotechar='"', on_bad_lines=False)

In [ ]:
# Modifying columns to lower case
df1.rename(columns={i:i.lower() for i in df1.columns}, inplace=True)

In [ ]:
df1.rename(columns={'operator_in_commercial_name':'operator_in'}, inplace=True)

In [ ]:
# Checking dataframe info
pd.DataFrame(df1.info())

## Cleaning Data

In [ ]:
#pd.concat(pd.DataFrame(df1.dtypes),df1.count())
df1.dtypes
#pd.concat(df1.dtypes, df1.count())
#df1.count()
#pd.concat()

In [ ]:
# Parsing object column to datetime
#df1['inserted_date'] = pd.to_datetime(df1['inserted_date']) #format='m%/%d/%Y %I:%M:%S %p')

date_col = 'order_updated' #-> Change to the desired column
def parse_datetime(date_col):
  df1[date_col] = pd.to_datetime(df1[date_col], format='%Y-%m-%d %H:%M:%S.%f')

parse_datetime(date_col)

In [ ]:
# Extracting year, month, day of the week, hour and date categories

date_col = 'timestamp' #-> Change to the desired column
def extract_datetime(df, date_col):
  df = df.assign(year = df[date_col].dt.year,
                 month = df[date_col].dt.month_name(),
                 weeknum = df[date_col].dt.isocalendar().week,
                 week_of_year = 'w' + df[date_col].dt.isocalendar().week.astype(str),
                 day_of_week = df[date_col].dt.day_name(),
                 hour = df[date_col].dt.hour.astype(str) + 'h',
                 hour_of_day = df[date_col].dt.hour)
  return df

df1 = extract_datetime(df1, date_col)

In [ ]:
# Extracting latitud and longitud from city names
def extract_lat_and_lon(df):
  geolocator = Nominatim(user_agent="MyApp")
  cities = sorted([i for i in set(df1['municipality'])])
  ci_lat = {}
  ci_lon = {}
  for i in cities:
    location = geolocator.geocode(i)
    ci_lat.update({i:location.latitude})
    ci_lon.update({i:location.longitude})
  #ci_lat = {i: geolocator.geocode(i).latitude for i in cities}
  #ci_lon = {i: geolocator.geocode(i).longitude for i in cities}
  df = df.assign(lat = df['municipality'].replace(to_replace=ci_lat),
                 lon = df['municipality'].replace(to_replace=ci_lon)
                 )
  return df
  #return {i: (geolocator.geocode(i).latitude, geolocator.geocode(i).longitude) for i in my_list}

df1 = extract_lat_and_lon(df1)
df1.tail()

In [ ]:
# Cleaning invalid orgnum
org_col = 'customer_orgnumber' #-> Change to the desired column
def check_valid_org_num(df):
  df = df.assign(org_check = np.where(df['org_col'].isnull(), 'Wrong',
                             np.where(df['org_col'].str.len()>9, 'Wrong',
                             np.where(df['org_col'].str.match(r'\d{9}'),'Valid','Wrong'))))
  return df

df1 = check_valid_org_num(df1)

In [ ]:
# Flattening list of list and extracting unique valid IDs from column
def extract_valid_ids(df, column_name):
  temp1 = [list(json.loads(i).items()) for i in df[column_name]]
  temp2 = [i for x in temp1 for i in x]
  temp3 = [i[1] for i in temp2 if i[0] == 'naviIdIfMinBedrift']
  flat = list(set(temp3))
  f_df = pd.DataFrame(flat, columns=['valid_id']).sort_values(by='valid_id').reset_index(drop=True)
  return f_df
  #flat = [i for x in temp1 for i in x]
  #set(flat)

df2 = extract_valid_ids(df1,'custom_dimensions')
df2.head()

In [ ]:
# Transforming nace code
df1 = df1.assign(market_nace_main_desc = df1['nace_main_desc'].str[:25])

In [ ]:
# Slicing county
df1 = df1.assign(county = df1['county'].str[:16])

In [ ]:
# Assigning brand preference
df1 = df1.assign(brand = np.where(df1['terminal_name'].str.contains('iPhone'),'Apple',
                                  np.where(df1['terminal_name'].str.contains('Samsung'),'Samsung', 'Other')))

In [ ]:
# Creating categorical orgnum column
df1 = df1.assign(org_num = 'ORG' + df1['organization_number'].astype(str))

In [ ]:
# Creating time of day column
df1 = df1.assign(time_of_day = np.where(df1['hr']<=11,'Morning',np.where(df1['hr']<=14,'Lunch','Afternoon')))

In [ ]:
# Customizing terminal categories
df1 = df1.assign(product_choice = np.where(df1['product_name'].str.contains('iPhone 11'),'iPhone 11',
                                           np.where(df1['product_name'].str.contains('iPhone 12'),'iPhone 12',
                                                    np.where(df1['product_name'].str.contains('iPhone 13'),'iPhone 13',
                                                             np.where(df1['product_name'].str.contains('iPhone 14'),'iPhone 14',
                                                                      np.where(df1['product_name'].str.contains('OnePlus'),'OnePlus',
                                                                               'Samsung'))))))

In [ ]:
# Checking the dataframe tail
df1.tail(1)

In [ ]:
df1['terminal_5g_device'] = df1['terminal_5g_device'].astype('str')

In [ ]:
# Selecting categorical and numeric columns for EDA
custom_list_1 = df1.select_dtypes(include=['object']).copy().columns.tolist()
custom_list_2 = [i for i in df1.select_dtypes(include=['float64','int64']).copy().columns.tolist() if '_id' not in i]

In [ ]:
[i for i in df1.select_dtypes(include=['float64','int64']).copy().columns.tolist() if '_id' not in i]

In [ ]:
"-".join([i[0:2] for i in df1['nace_main'][0].split(' ')])

In [ ]:
df1[df1['org_check']=='Valid'][['inserted_date','customer_orgnumber','org_check']].reset_index()

In [ ]:
# Temp1: Slicing dataframe 
df2 = df1[['orderid','statustext','username','listname','inserted_date','customer_orgnumber','customer_company',
'customer_phone','customer_cellphone','customer_email','accept_date','productname','nummer som skal benytte',
'orderproductsquantity','productquantity','year','month','weeknum','day','hour','hr','org_check']].copy()
df2 = df2[df2['org_check']=='Valid'].reset_index(drop=True)

In [ ]:
df1.rename( columns={'client_country_or_region':'country'}, inplace=True)

In [ ]:
df1['custom_dimensions'][5]

In [ ]:
df2 = df1[(df1['loyalty_flag']=='Yes')].reset_index(drop=True).copy()

In [ ]:
df2.tail()

## Exploring Data

In [ ]:
# Displaying less rows in aggregations
pd.options.display.max_rows = 15
pd.options.display.min_rows = 10

In [ ]:
# Exploring list of categorical columns with value counts
#segm_list = ['Smartphone','Mobile broadband','Machine to machine']
@interact(Column_name=custom_list_1, Percentage=[True, False])
def explore_value_counts(Column_name, Percentage):
  if Percentage == True:
    df = df1.value_counts(subset=[Column_name], normalize=Percentage, dropna=False).reset_index(name='percent')
    df = df.assign(percent = round(df['percent'],6)*100)
    df = df.assign(cum_perc = df['percent'].cumsum())
    df = df.assign(percent = df['percent'].round(2).astype(str) + '%')
    df = df.assign(cum_perc = df['cum_perc'].round(2).astype(str) + '%')
  else:
    df = df1.value_counts(subset=[Column_name], normalize=Percentage, dropna=False).reset_index(name='events')
    #df = df.sort_values(by=['events']).reset_index(drop=True)
  return df

In [ ]:
# Exploring list of categorical columns with value counts
@interact(Categories=custom_list_1, Column_name=custom_list_2, Aggregate=['count','nunique','sum','mean','max','min'])
def explore_numeric_columns(Categories, Column_name, Aggregate):
  df = df1[df1['name']=='Performed'].groupby(by=Categories).agg(value=(Column_name,Aggregate)).reset_index()
  df.rename(columns={'value':Column_name}, inplace=True)
  df = df.sort_values(by=[Column_name],ascending=False).reset_index(drop=True)
  return df

In [ ]:
# Plotting categorical columns with numeric columns
agg_list = ['sum','mean','median','max','min','nunique','count']
#segm_list = ['Smartphone','Mobile broadband','Machine to machine']
@interact(Categories=['weeknum'], Column_name=custom_list_2[1:], Aggregate=agg_list)
def visualize_two_columns(Categories, Column_name, Aggregate):
  df = df1.groupby(by=[Categories]).agg(value=(Column_name,Aggregate)).reset_index()
  df.rename(columns={'value':Column_name}, inplace=True)
  df = df.sort_values(by=[Column_name],ascending=False).reset_index(drop=True)
  #return df
  fig_x, axes = plt.subplots(1, 1, figsize=(9.0, 4.0))
  axes.set_title(f'MinBedrift - {Aggregate} of {Categories}')
  sns.barplot(x=Categories, y=Column_name, data=df, estimator=Aggregate, order=[i for i in range(1,14)], color ='#0080FF',ax=axes)
  for i in axes.containers:
    axes.bar_label(i,fmt='%.f')
  #plt.legend(title='Is_Loyal', bbox_to_anchor=(1.15, 1), loc=2, borderaxespad=0.)
  plt.show()

In [ ]:
# Visualizing histogram of numeric columns dynamically
segm_list = ['Smartphone','Mobile broadband','Machine to machine']
dis_list_1 =['count', 'frequency','probability', 'proportion', 'percent', 'density']
@interact(Family=segm_list, Column=custom_list_2[2:], Graph_type=dis_list_1)
def plot_histogram(Family, Column, Graph_type):
  df = df1[df1['product_family']==Family].reset_index(drop=True).copy()
  Q1 = df[Column].quantile(0.25)
  Q3 = df[Column].quantile(0.75)
  IQR = Q3 - Q1
  df = df[(df[Column]>(Q1-1.5*IQR)) & (df[Column]<(Q3+1.5*IQR))].reset_index(drop=True)
  fig_x, axes = plt.subplots(1, 1, figsize=(7.5, 3.5))
  axes.set_title(f'{Family}: {Graph_type} - {Column}')
  sns.histplot(x=Column, data=df, hue='loyalty_flag', stat=Graph_type, bins=15, 
               multiple='stack', palette={'Yes':'green','No':'#e49444'}, ax=axes)
  for i in axes.containers:
    axes.bar_label(i,fmt='%.2f')
  plt.show()

In [ ]:
# Visualizing distribution of numeric columns dynamically
segm_list = ['Smartphone','Mobile broadband','Machine to machine']
dis_list_2 =['hist', 'kde','ecdf']
@interact(Family=segm_list, Column=custom_list_2[2:], Graph_type=dis_list_2, Log_sc=[False,True])
def plot_distribution(Family, Column, Graph_type, Log_sc):
  df = df1[df1['product_family']==Family].reset_index(drop=True).copy()
  Q1 = df[Column].quantile(0.25)
  Q3 = df[Column].quantile(0.75)
  IQR = Q3 - Q1
  df = df[(df[Column]>(Q1-1.5*IQR)) & (df[Column]<(Q3+1.5*IQR))].reset_index(drop=True)
  #sns.set(rc={'figure.figsize':(10,5)})
  sns.displot(x=Column, data=df, hue='loyalty_flag', kind=Graph_type, log_scale=Log_sc, #multiple='stack',
              palette={'Yes':'green','No':'#e49444'}, height=4.5, aspect=2) 
  plt.title(f'{Family} - {Column}')
  plt.axhline(y=0.5, color='red', ls='--')
  plt.show()

In [ ]:
# Visualizing distribution of numeric columns dynamically
segm_list = ['Smartphone','Mobile broadband','Machine to machine']
@interact(Family=segm_list, X_col=custom_list_2[2:], Y_col=custom_list_1)
def plot_boxplot(Family, X_col, Y_col):
  df = df1[df1['product_family']==Family].reset_index(drop=True).copy()
  Q1 = df[X_col].quantile(0.25)
  Q3 = df[X_col].quantile(0.75)
  IQR = Q3 - Q1
  df = df[(df[X_col]>(Q1-1.5*IQR)) & (df[X_col]<(Q3+1.5*IQR))].reset_index(drop=True)
  #sns.set(rc={'figure.figsize':(10,5)})
  sns.boxplot(x=X_col, y=Y_col, data=df, hue='loyalty_flag', palette={'Yes':'green','No':'#e49444'})
  plt.title(f'{X_col} - {Y_col}')
  plt.show()

In [ ]:
# Plotting pair plot of numeri variables 
segm_list = ['Smartphone','Mobile broadband','Machine to machine']
hue_list = ['loyalty_flag']
@interact(Family=segm_list, Column=custom_list_2[2:], Flag=hue_list)
def plot_pairplot(Family, Column, Flag):
  df = df1[df1['product_family']==Family].reset_index(drop=True).copy()
  df = df[['product_family','loyalty_flag','lifetime_days','mb_total_l3m_avg','voice_total_l3m_avg','subscriber_profit']].copy()
  sns.pairplot(data=df, hue=Flag)
  plt.show()

In [ ]:
# Visualizing scatterplot of numeric columns dynamically
@interact(Column=custom_list_2)
def visualize_numeric(Column):
  # 'count', 'frequency','probability', 'proportion', 'percent', 'density'
  Q1 = df1[Column].quantile(0.25)
  Q3 = df1[Column].quantile(0.75)
  IQR = Q3 - Q1
  df = df1[(df1[Column]>(Q1-1.5*IQR)) & (df1[Column]<(Q3+1.5*IQR))].reset_index(drop=True)
  #sns.set(rc={'figure.figsize':(10,5)})
  sns.scatterplot(x='lifetime_days', y=Column, data=df, hue='loyalty_flag', style='loyalty_flag') #color='green' #binwidth=2
  plt.title(f'Histogram - {Column}')
  plt.show()

In [ ]:
# Visualizing time columns
@interact(Time_column=['month','week_of_year','day_of_week','hour'])
def visualize_time_dimension(Time_column):
  sns.catplot(x=Time_column, data=df1, kind='count', color='#0080FF', height=5, aspect=2.0,
              order=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
              #order=['0h','1h','2h','3h','4h','5h','6h','7h','8h','9h','10h','11h','12h',
              #'13h','14h','15h','16h','17h','18h','19h','20h','21h','22h','23h']) #'#0080FF', '#070707', '#454545'
  )
  plt.title(f'Events by {Time_column}')
  #plt.axhline(y=43, color='red', label='avg_week')
  #plt.legend(bbox_to_anchor = (1.1, 1), loc = 'upper center')
  plt.show()

In [ ]:
# Interacting with list of columns
@interact(Column_name = custom_list_1, Variable = custom_list_2)
def visualize_nace(Nace, Variable):
  df = df1[df1['market_nace_main_desc']==Nace].copy()
  df = df.groupby(by=['market_nace_main_desc','county']).agg(avg = (Variable,'mean')).reset_index()
  fig_x, axes = plt.subplots(1, 1, figsize=(7.5, 5.5))
  axes.set_title(f'{Nace}: {Variable}')
  sns.barplot(x='market_nace_main_desc', y='avg', data=df, hue='county', palette='tab20', ax=axes)
  plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  plt.show()

## Aggregating Data

In [ ]:
# Selecting columns for aggregation
custom_list_2 = ['', '', '', ''] # => Add column names for slicing here.

In [ ]:
# Main data aggregation
df1_a = df1.groupby(by=custom_list_2, dropna=False).agg(CHURN_COUNT = ('CUSTOMER_ID', 'count')).reset_index() # => Note: Dropna=False to avoid dropping data if group keys contain NA values.

In [ ]:
# Creating list of aggregated fields
df1_list = [(pd.DataFrame(df1[i].value_counts(dropna=False))) for i in custom_list_1]

## Exporting results to local drive

In [ ]:
# Exporting main excel file
with pd.ExcelWriter('script_05.xlsx', engine='openpyxl') as writer:
  df2.to_excel(writer, sheet_name='Sheet1', index=False)
files.download('script_05.xlsx')

In [ ]:
# Exporting list of excel sheets
with pd.ExcelWriter('Script_202208221130.xlsx', engine='openpyxl') as writer:
  for i in range(len(df1_list)-1):
    df1_list[i].to_excel(writer, sheet_name=f'{df1_list[i].columns[0]}', index=True)
files.download('Script_202208221130.xlsx')